In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer

from keras.preprocessing import sequence
from keras.preprocessing.sequence import  pad_sequences
from keras.datasets import imdb

from konlpy.tag import Okt
import statsmodels.api as sm
%matplotlib inline

Using TensorFlow backend.


In [2]:
train_data= pd.read_table('data/ratings_train.txt')
test_data= pd.read_table('data/ratings_test.txt')

#전처리를 안하면 점수가 잘 안나오기 때문에 전처리 수행
train_data=train_data.dropna(how='any') # Null 값이 존재하는 행 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] #특별한 워드 설정

In [3]:
okt = Okt()
X_train=[]
for sentence in train_data['document']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용단어 제거, 한글 단어만 들어가있도록 설정
    X_train.append(temp_X)   

In [4]:
test_data=test_data.dropna(how='any') # Null 값 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행

X_test=[]
for sentence in test_data['document']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [5]:
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존, 텍스트를 시퀀스(인덱스)로 변형, 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [6]:
max_len=30
# 전체 데이터의 길이는 30으로 맞춘다.
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
y_train=np.array(train_data['label']) #
y_test=np.array(test_data['label']) #긍정, 부정 의 0 , 1 레이블 들어가있음

In [7]:
model = Sequential()
model.add(Embedding(max_words, 100))  # 100개의 차원
# 워드 임베딩이 구현되어 있다. 전체 데이터 길이를  동일하게 설정, 트레인과 테스트가 동일하게 설정 됨
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid')) # 긍정, 또는 부정을 출력하므로 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
history = model.fit(X_train, y_train, epochs=2, batch_size=60, validation_split=0.2)  # 1번 2분  84% -> 85%


Train on 119996 samples, validate on 29999 samples
Epoch 1/2
119996/119996 [==============================] - 117s 976us/step - loss: 0.3871 - accuracy: 0.8221 - val_loss: 0.3479 - val_accuracy: 0.8455
Epoch 2/2
119996/119996 [==============================] - 118s 979us/step - loss: 0.2896 - accuracy: 0.8760 - val_loss: 0.3426 - val_accuracy: 0.8516


In [9]:
# 9:48
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

49997/49997 [==============================] - 6s 127us/step

 테스트 정확도: 0.8460


In [11]:
model.evaluate(X_test, y_test)

49997/49997 [==============================] - 6s 122us/step


[0.35574681900491867, 0.8460107445716858]